In [1]:
import requests
import re
import time
from bs4 import BeautifulSoup
import datetime
import collections
from typing import List, Dict, Union, Any, Generator
import csv
import json
import jsonlines
import random
import pytz

In [2]:
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Safari/605.1.15',
    'Cookie': 'cycada=bPpdsL6OmVuA/vHwOkwEZ6b5m1yT1mYEBUWGwH8FGDg=; _ym_d=1576847554; _ym_uid=1570718328767985713; _ym_visorc_22663942=b; _ym_visorc_52332406=b; mda_exp_enabled=1; noflash=true; sso_status=sso.passport.yandex.ru:synchronized_no_beacon; tc=1; user_country=ru; yandex_gid=213; yandex_plus_metrika_cookie=true; PHPSESSID=jsb8usnpa8ovgsjtjp9gg10kk3; mobile=no; _csrf_csrf_token=uDZ_fZZywgrIXC2KLpV2V6G_K71VsmISBCwNr0heC4U; _ym_isad=2; _ym_wasSynced=%7B%22time%22%3A1576844395295%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; mda=0; desktop_session_key=139f10f99471995a1b48eecf0b98f5888c9219d00a10771efaba78cabc3dbaaeecd4f9dfd069f86f3ebf8f6b3a2bfb97bd95a5a3387a5bf37859968e4d756ffafaf00965312aa4f2cc93c262df127542eb204f0518e55383ed78b02c0dce7e36; desktop_session_key.sig=dP943b2vDA3HinOlIUvCc8FOVBs; user-geo-country-id=2; user-geo-region-id=213; lfiltr=all; i=q5DjteMdDtdbTPiy8XS1deq7n8vXuonLoQeLibMR86Zz/u+f0CVJZDEJHTndsyoTjK9LBCz3DwQ8WWr5hbXDZSmu60I=; mda2_beacon=1575744883269; ya_sess_id=3:1575744883.5.1.1568026663610:lXu8Lg:1d.1|924553428.-1.0.1:201533161|894704379.-1.0.1:191832246.2:1397019|30:185633.95409.RVP7okPx9W_5Mm-mpZ96xd48jxs; yandex_login=uid-ledkxpyj; yandexuid=5418682411560849018; my_perpages=%7B%2260%22%3A200%7D; _ym_d=1574701806; _ym_uid=1570718328767985713',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Encoding': 'br, gzip, deflate',
    'Accept-Language': 'ru'
})

In [3]:
def get_movie_info(number: int, title: str,
                   kp_rating: float, imbd_rating: float,
                   year: int, url: str, utc_time: str) -> Dict[str, Any]:
    movie_info = {}
    movie_info['number'] = number
    movie_info['title'] = title
    movie_info['kp_rating'] = kp_rating
    movie_info['imbd_rating'] = imbd_rating
    movie_info['year'] = year
    movie_info['url'] = url
    movie_info['_utc_timestamp'] = utc_time
    return movie_info

In [4]:
def movies_info_generator(max_amount: int = 17,
                          delay: Union[int, float] = 0.5) -> Generator [Dict[str, Any], None, None]:
    if delay < 0:
        raise ValueError('delay must not be negative')
    d = 0
    for i in range(3):
        now_utc = str(datetime.datetime.utcnow().replace(microsecond=0))
        resp = session.get('https://www.kinopoisk.ru/top/lists/1/filtr/all/sort/order/page/%s/' % str(i+1)).text
        time.sleep(delay)
        soup = BeautifulSoup(resp)
        for k in range(200):
            if d+1 > max_amount:
                break
            try:
                number = soup.find('div', class_ = 'num rangImp', string = d+1) # тэг с номером по сортировке
                url_movie_info = number.parent.parent # кусок кода страницы по 1 фильму
                movie_number = number.previous_sibling.previous_sibling['id'] # уникальный номер фильма 'film___'
                movie_title = url_movie_info.find('a', class_ = 'all').string # название фильма
                kp_rating = float(url_movie_info.find('div', attrs ={'data-film-rating': True})['data-film-rating']) # рейтинг кинопоиска
                imbd_rating = float(url_movie_info.find('small').previous_element[6:]) # рейтинг imbd
                movie_year = re.search('\d{4}', url_movie_info.find('span', style = True).next_element).group() # год фильма
                movie_url = 'https://www.kinopoisk.ru/film/' + movie_number[4:] # ссылка на фильм
                movie_info = get_movie_info(d+1, movie_title, kp_rating, imbd_rating, movie_year, movie_url, now_utc)
                yield movie_info
                d += 1
            except AttributeError:
                break

In [5]:
movie_list = list(movies_info_generator(500))

In [6]:
sorted_list = sorted(movie_list, key=lambda imbd: imbd['imbd_rating'], reverse = True)

In [7]:
years_list = [movie['year'] for movie in movie_list]
c = collections.Counter(years_list)
print('Самый популярный год: ' + str(c.most_common(1)[0][0]))
print('Минимальный год: ' + str(min(c))  + '\nМаксимальный год: ' + str(max(c)))

Самый популярный год: 2004
Минимальный год: 1921
Максимальный год: 2019


In [8]:
def dump(film_list: List[Dict[str, Any]], 
         path: str, 
         filetype: str = 'csv', 
         order_by: Union[None, str] = None, # поле словарей, по которому будем сортировать
         ascending: bool = True) -> None:
    columns = ['number', 'title', 'kp_rating', 'imbd_rating', 'year', 'url', '_utc_timestamp']
    if filetype not in ['csv', 'tsv', 'json', 'jsonl']:
        raise ValueError('wrong filetype, available filetypes: csv, tsv, json, jsonl')
    if order_by not in columns:
        raise ValueError('wrong value, may be ordered by: number, title, kp_rating, '
                         'imbd_rating, year, url, _utc_timestamp')
    if filetype in ['csv', 'tsv']:
        sep = ';' if filetype =='csv' else '\t'
        with open('myfile.' + filetype, 'w', encoding='utf-8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns, delimiter=sep)
            writer.writeheader()
            writer.writerows(sorted(film_list, key=lambda ord_by: ord_by [order_by], reverse= not ascending))
    elif filetype == 'json':
        with open ('my_file.json', 'w') as f:
            json.dump(sorted(movie_list, key=lambda ord_by: ord_by [order_by], reverse= not ascending),
                      f, indent=4, ensure_ascii=False,)
    elif filetype =='jsonl':
        with jsonlines.open('my_file.jsonl', 'w') as f:
            f.write_all(sorted(movie_list, key=lambda ord_by: ord_by [order_by], reverse= not ascending))
    

dump(movie_list, '/Users/mak/Desktop/Учеба/3 курс/proga/', filetype='csv', order_by='title', ascending=False)

In [9]:
def rand_IP():
    IP = '.'.join([str(random.randint(0, 255)) for i in range(4)])
    return IP

In [10]:
print('Случайный IPv4-адрес: ' + str(rand_IP()))

Случайный IPv4-адрес: 17.154.214.114


In [11]:
IP_hundred = [] # список из 100 разных IP-адресов
while len(IP_hundred) <100:
    IP = '.'.join([str(random.randint(0, 255)) for i in range(4)])
    if IP not in IP_hundred:
        IP_hundred.append(IP)

In [12]:
headers = {
    'accept': "application/json",
    'content-type': "application/json"
    }
IP_hundred_info = [] # информация о 100 рандомных IP-адресах в формате JSON
for IP in IP_hundred:
    url = 'https://freegeoip.app/json/{}'.format(IP)
    response = requests.request("GET", url, headers=headers)
    time.sleep(random.uniform(0.5, 1))
    IP_info = json.loads(response.text)
    IP_hundred_info.append(IP_info)

In [13]:
start_date = datetime.datetime(2019, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
start_date_timestamp = start_date.timestamp()
end_date = datetime.datetime(2019, 12, 31, 23, 59, 59, tzinfo=datetime.timezone.utc)
end_date_timestamp = end_date.timestamp()
year_diff = end_date_timestamp - start_date_timestamp
rand_moment_timestamp = start_date_timestamp + random.randint(0, year_diff)
rand_moment = datetime.datetime.fromtimestamp(rand_moment_timestamp, tz =datetime.timezone.utc)
print('Случайный момент времени в 2019 году в виде unix timestamp (по UTC времени): ' + str(rand_moment))

Случайный момент времени в 2019 году в виде unix timestamp (по UTC времени): 2019-05-01 05:15:32+00:00


In [14]:
unique_timezones = []
for IP_info in IP_hundred_info:
    if IP_info['time_zone'] != '' and IP_info['time_zone'] not in unique_timezones:
        unique_timezones.append(IP_info['time_zone'])
print('Этот случайный момент, приведенный к встретившимся часовым поясам')
for timezone in unique_timezones:
    print(str(rand_moment.astimezone(pytz.timezone(timezone))) + '\t' + str(timezone))

Этот случайный момент, приведенный к встретившимся часовым поясам
2019-05-01 00:15:32-05:00	America/Chicago
2019-05-01 07:15:32+02:00	Europe/Berlin
2019-05-01 13:15:32+08:00	Asia/Shanghai
2019-05-01 13:15:32+08:00	Asia/Taipei
2019-05-01 07:15:32+02:00	Europe/Amsterdam
2019-05-01 13:15:32+08:00	Asia/Hong_Kong
2019-05-01 07:15:32+02:00	Europe/Copenhagen
2019-05-01 06:15:32+01:00	Africa/Casablanca
2019-05-01 06:15:32+01:00	Europe/London
2019-05-01 07:15:32+02:00	Africa/Johannesburg
2019-05-01 14:15:32+09:00	Asia/Tokyo
2019-05-01 08:15:32+03:00	Europe/Kiev
2019-05-01 07:15:32+02:00	Europe/Madrid
2019-05-01 08:15:32+03:00	Europe/Athens
2019-05-01 07:15:32+02:00	Europe/Paris
2019-05-01 08:15:32+03:00	Africa/Kampala
2019-04-30 22:15:32-07:00	America/Los_Angeles
2019-05-01 07:15:32+02:00	Europe/Zurich
2019-05-01 10:15:32+05:00	Asia/Yekaterinburg
2019-05-01 01:15:32-04:00	America/Toronto
2019-05-01 00:15:32-05:00	America/Bogota
2019-05-01 02:15:32-03:00	America/Sao_Paulo
2019-05-01 07:15:32+02: